In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
%matplotlib inline

# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

In [2]:
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
#1.產生一個新的變數 Survived_cate ，資料型態傳換成類別型態      
#2.把題目中的 Survived 用 Survived_cate 來做分析    

df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [4]:
#我們先把遺失值刪除
##取出資料後，把遺失值刪除
complete_data=df_train.dropna().reset_index(drop='bool')

#排除 Ｎame 當特徵，先刪除每一個人都是獨特的資料，先檢視其他變數
complete_data=complete_data.drop(['Name','Ticket','PassengerId'], axis=1)
display(complete_data.head(5))

print(complete_data.shape)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate
0,1,1,female,38.0,1,0,71.2833,C85,C,1
1,1,1,female,35.0,1,0,53.1000,C123,S,1
2,0,1,male,54.0,0,0,51.8625,E46,S,0
3,1,3,female,4.0,1,1,16.7000,G6,S,1
4,1,1,female,58.0,0,0,26.5500,C103,S,1


(183, 10)


In [5]:
num_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : \n{num_features}\n')

cat_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'object':
        cat_features.append(feature)
print(f'{len(cat_features)} category Features : \n{cat_features}')

6 Numeric Features : 
['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

4 category Features : 
['Sex', 'Cabin', 'Embarked', 'Survived_cate']


In [6]:
'''
Q1: 目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，
試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。
'''

#離散要轉換成數值，['Sex', 'Embarked']
for_sex = {'female':0, 'male':1}
for_embarked = {'C':0, 'S':1, 'Q':2}

complete_data['Sex_0'] = complete_data['Sex'].map(for_sex)
complete_data['Embarked_0'] = complete_data['Embarked'].map(for_embarked)

#執行 包裝法
x = complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex_0', 'Embarked_0']]
y = complete_data['Survived']
display(x.head(5))
display(y.head(5))

estimator = SVC(kernel='linear') #Survived為離散型
selector = RFE(estimator)
selector = selector.fit(x, y)
print(selector.support_)
print(selector.ranking_)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)

,Pclass,Age,SibSp,Parch,Fare,Sex_0,Embarked_0
0,1,38.0,1,0,71.2833,0,0
1,1,35.0,1,0,53.1000,0,1
2,1,54.0,0,0,51.8625,1,1
3,3,4.0,1,1,16.7000,0,1
4,1,58.0,0,0,26.5500,0,1


0    1
1    1
2    0
3    1
4    1
Name: Survived, dtype: int64

[False False  True False False  True  True]
[2 4 1 3 5 1 1]
['SibSp', 'Sex_0', 'Embarked_0']


In [10]:
'''
重要性為
1:'SibSp', 'Sex_0', 'Embarked_0'
2:'Pclass'
3:'Parch'
4:'Age'
5:'Fare'

其中，建議選擇排名為1的特徵做為分析項目
'''

"\n重要性為\n1:'SibSp', 'Sex_0', 'Embarked_0'\n2:'Pclass'\n3:'Parch'\n4:'Age'\n5:'Fare'\n\n其中，建議選擇排名為1的特徵做為分析項目\n"